<a href="https://colab.research.google.com/github/addis0nl/AutoStoryteller/blob/main/gradioUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Download/Install

In [1]:
! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78
! pip install tts
! pip install diffusers
! pip install ebooklib
! pip install gradio
! mkdir outputs
! mkdir outputs/tts
! mkdir outputs/txt2img
! mkdir voices
! wget -P voices/ https://edresson.github.io/YourTTS/audios_demo/reference/p225_005.wav
! wget -nc https://huggingface.co/TheBloke/AlpacaCielo2-7B-8K-GGML/resolve/main/alpacacielo2-7b-8k.ggmlv3.q4_K_S.bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822288 sha256=3fe4f455ecc3b45fdfdec4b6a9499fe52b156d94e4e15d54aa989621a8bd24a3
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.3/809.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.1/989.1 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.20.2-py3-none-any.whl size=1342633 sha256=53c919258d1937e4e560f41cb6eae6336ef3afcade53bfa725a79c87df82ec59
  Stored in directory: /root/.cache/pip/wheels/dc/8b/d9/34f7a1936109e05e9bba0cc2241a6f8cd89e25959dc7aae942
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=c3c7d67a47addc61924d573c599deb0327bc312e41b19b063152fa1279ffda09
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 55.4 MB/s eta 0:00:0

##### Stable Diffusion

In [2]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, revision='fp16', torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

def sd21(prompt=None):

    image = pipe(prompt, num_inference_steps=20, height=512).images[0]
    print("Prompt -",prompt,"- complete.")
    return image

text_encoder/model.safetensors not found


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

##### Llama & Coqui

In [ ]:
from llama_cpp import Llama

path = "alpacacielo2-7b-8k.ggmlv3.q4_K_S.bin"     # Change if desired
llm = Llama(model_path=path, n_gpu_layers=-1, seed=-1, n_ctx=4096)

def textgen(prompt):
    output = llm("Q: " + prompt + " \n\nA: \n", max_tokens=4096, stop=["Q:"], echo=False)
    text = output['choices'][0]['text']
    tokeninfo = output['usage']
    # print(text, "\n")
    print(tokeninfo)
    return text

In [4]:
from TTS.api import TTS
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=True, gpu=True)
def yourtts(text, sample, outpath):
    tts.tts_to_file(text, speaker_wav=sample, language="en", file_path=outpath)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:77: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


100%|██████████| 425M/425M [00:11<00:00, 35.6MiB/s]


 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | 

##### Gradio GUI

In [ ]:
import gradio as gr
import ebooklib
from bs4 import BeautifulSoup
import re
import subprocess
import numpy as np
import os
from datetime import datetime

def get_chapters(filename):
    book = ebooklib.epub.read_epub(filename)
    title = book.title.replace(' ', '')
    chapters = []
    count = 0
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        item_id = item.get_id()
        if re.match(".*c[h012]", item_id):
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')
            text = [para.get_text() for para in soup.find_all(text=True)]
            chapters.append((title + '_' + str(count), '   '.join(text)))
            count += 1
    return chapters

def load_text(file):
    chapters = get_chapters(file.name)
    return chapters, chapters[0][1], 0, gr.update(open=False), gr.update(visible=True), gr.update(visible=True)

def load_audio(list, voice, page):
    page = int(page['label'])
    paragraphs = (list[page][1].split('\n'))
    texts = np.array_split(paragraphs, 2)
    audio_files = []
    text = (''.join(texts[0]), ''.join(texts[1]))
    outnameA = 'outputs/tts/' + list[page][0] + '_a_' + voice
    outnameB = 'outputs/tts/' + list[page][0] + '_b_' + voice
    audio_files.append(outnameA[12:])
    audio_files.append(outnameB[12:])
    if not os.path.isfile(outnameA):
        yourtts(text[0], "voices/" + voice, outnameA)
        yourtts(text[1], "voices/" + voice, outnameB)
    return gr.update(choices=audio_files)

def next(list, page):
    page = int(page['label'])
    if page == list[-1][0]:
        return list[0][1], 0
    return list[page + 1][1], page + 1

def prev(list, page):
    page = int(page['label'])
    if page == list[-1][0] * -1:
        return list[0][1], 0
    return list[page - 1][1], page - 1

def play(file):
    return 'outputs/tts/' + file

def clear():
    return gr.update(value="")

def draw(prompt):
    if prompt != "":
        img = sd21(prompt)

        folder = sorted(os.listdir("outputs/txt2img"))
        if not folder:
            num = 1
        else:
            num = int(folder[-1][:-4]) + 1
        num = f"{num:03d}"
        img_dir = "outputs/txt2img/" + num + ".png"
        img.save(img_dir)
        return img_dir
    else:
      return

def writecustom(a, b, c):
    if a == "":
        return "", [(None, None)], None, None, gr.update(open=False), gr.update(visible=True), gr.update(visible=True)
    instr = strA + a
    if b != "":
        instr += strB + b
    if c != "":
        instr += strC + c
    return write(instr)

def write(prompt=""):
    if prompt == "":
        prompt = "Using the first sentence to describe the scenery, write a long story"
    out = textgen(prompt)
    opening = out.split(".")[0]
    pic = draw(opening)
    now = datetime.now()
    id = now.strftime("%Y%m%d%H%M")
    return out, [(id, out)], '0', pic, gr.update(open=False), gr.update(visible=True), gr.update(visible=True)

In [ ]:
strA = "Write a story about..."
strB = "...set in..."
strC = "...where..."

with gr.Blocks(theme='finlaymacklon/smooth_slate') as demo:
    with gr.Row():
        with gr.Column():
            section1 = gr.Accordion(label="Start")
            section2 = gr.Accordion(label="Enhancements", visible=False)

            with section1:
                with gr.Tab("Random"):
                    gr.Markdown(
                        """
                        # AutoStoryteller
                        Harnessing various generative models to create an automated picture book experience.
                        Made for entertainment purposes only. Please allow for approximately 4 minutes of runtime.
                        <br><br>
                        To start, click the button below for a random story, select the 'custom' tab above to add
                        your own ideas, or select the 'upload' tab to import from a .epub file (note: will not
                        auto gen an image).
                        <br><br>
                        Text generation via 'dolphin-llama2-7b' LLM.
                        Image generation via Stable Diffusion 2.1.
                        Voice synthesis via YourTTS.
                        <br>
                        """)
                    auto = gr.Button("Tell me a story").style(full_width=False)
                with gr.Tab("Custom"):
                    a = gr.Textbox(label=strA, placeholder="subject")
                    b = gr.Textbox(label=strB, placeholder="place and/or time (optional)")
                    c = gr.Textbox(label=strC, placeholder="extra context (optional)")
                    write_btn = gr.Button("Proceed").style(full_width=False)
                with gr.Tab("Upload"):
                    file = gr.File(label=".epub files only")

            with section2:
                with gr.Tab("Image"):
                    image = gr.Image(show_label=False, interactive=False).style(height=467)
                    s = "Copy and paste sentences here to generate a image to compliment the text, ~30s runtime"
                    prompt = gr.Textbox(show_label=False, placeholder=s, lines=2,
                                        max_lines=2)
                    with gr.Row():
                        submit_btn = gr.Button("Submit")
                        clear_btn = gr.Button("Clear")

                with gr.Tab("Audio"):
                    gr.Markdown("Select a voice and click the button to generate speech of the displayed text, "
                                "~60s runtime.")
                    with gr.Row():
                        voice = gr.Dropdown(os.listdir("voices/"), label="Voice", value=os.listdir("voices/")[0],
                                            interactive=True)
                        load_audio_btn = gr.Button("Load audio").style(full_width=False)
                    audio = gr.Radio(label="Audio", interactive=True)
                    player = gr.Audio(label="Player", interactive=False)

        with gr.Column():
            section3 = gr.Column(visible=False)
            with section3:
                output = gr.Textbox(show_label=False, lines=41, max_lines=41)
                with gr.Row():
                    prev_btn = gr.Button("Prev chapter")
                    next_btn = gr.Button("Next chapter")

    page = gr.Label(visible=False)
    list = gr.List(visible=False)
    file.upload(fn=load_text, inputs=file, outputs=[list, output, page, section1, section2, section3])
    load_audio_btn.click(fn=load_audio, inputs=[list, voice, page], outputs=audio)
    audio.change(fn=play, inputs=audio, outputs=player)
    next_btn.click(fn=next, inputs=[list, page], outputs=[output, page])
    prev_btn.click(fn=prev, inputs=[list, page], outputs=[output, page])
    clear_btn.click(fn=clear, outputs=prompt)
    submit_btn.click(fn=draw, inputs=prompt, outputs=image)
    write_btn.click(fn=writecustom, inputs=[a, b, c], outputs=[output, list, page, image, section1, section2, section3])
    auto.click(fn=write, outputs=[output, list, page, image, section1, section2, section3])

##### Launch

In [7]:
demo.close()
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
